In [2]:
!wc -l ../tokenizers/uralic.txt

 9758237 ../tokenizers/uralic.txt


In [1]:
from tokenizer import load_vocab

In [19]:
tokenizer_files = {
    "estonian": "../tokenizers/wikipedia_et_all_nopic_2024-05.json",
    "finnish": "../tokenizers/wikipedia_fi_all_nopic_2023-05.json",
    "hungarian": "../tokenizers/wikipedia_hu_all_nopic_2024-05.json",
    "se": "../tokenizers/wikipedia_se_all_nopic_2024-05.json",
    "uraliv": "../tokenizers/uralic.json",
}

In [20]:
vocabs = {k: load_vocab(v) for k, v in tokenizer_files.items()}

100%|██████████| 256000/256000 [00:00<00:00, 534076.62it/s]


In [21]:
all_tokens = set()
for k, v in vocabs.items():
    for token in v:
        all_tokens.add(token)
len(all_tokens)

762307

In [22]:
token_to_ranks = {}
for i, (k, v) in enumerate(vocabs.items()):
    print(k, len(v))
    for token in v:
        token_to_ranks.setdefault(token, [0] * len(vocabs))
        token_to_ranks[token][i] = v[token] + 1
len(token_to_ranks)

estonian 256000
finnish 256000
hungarian 256000
se 104921
uraliv 256000


762307

In [24]:
for i, token in enumerate(token_to_ranks):
    if i > 300:
        break
    print(token, token_to_ranks[token])

<s> [1, 1, 1, 1, 1]
<pad> [2, 2, 2, 2, 2]
</s> [3, 3, 3, 3, 3]
<unk> [4, 4, 4, 4, 4]
<mask> [5, 5, 5, 5, 5]
! [6, 6, 6, 6, 6]
" [7, 7, 7, 7, 7]
# [8, 8, 8, 8, 8]
$ [9, 9, 9, 9, 9]
% [10, 10, 10, 10, 10]
& [11, 11, 11, 11, 11]
' [12, 12, 12, 12, 12]
( [13, 13, 13, 13, 13]
) [14, 14, 14, 14, 14]
* [15, 15, 15, 15, 15]
+ [16, 16, 16, 16, 16]
, [17, 17, 17, 17, 17]
- [18, 18, 18, 18, 18]
. [19, 19, 19, 19, 19]
/ [20, 20, 20, 20, 20]
0 [21, 21, 21, 21, 21]
1 [22, 22, 22, 22, 22]
2 [23, 23, 23, 23, 23]
3 [24, 24, 24, 24, 24]
4 [25, 25, 25, 25, 25]
5 [26, 26, 26, 26, 26]
6 [27, 27, 27, 27, 27]
7 [28, 28, 28, 28, 28]
8 [29, 29, 29, 29, 29]
9 [30, 30, 30, 30, 30]
: [31, 31, 31, 31, 31]
; [32, 32, 32, 32, 32]
< [33, 33, 33, 0, 33]
= [34, 34, 34, 33, 34]
> [35, 35, 35, 34, 35]
? [36, 36, 36, 35, 36]
@ [37, 37, 37, 0, 37]
A [38, 38, 38, 36, 38]
B [39, 39, 39, 37, 39]
C [40, 40, 40, 38, 40]
D [41, 41, 41, 39, 41]
E [42, 42, 42, 40, 42]
F [43, 43, 43, 41, 43]
G [44, 44, 44, 42, 44]
H [45, 45, 45, 43

In [26]:
def escape_token(token):
    return token.replace('\t', '\\t').replace('\r', '\\r').replace('\n', '\\n')

with open("../tokenizers/token_ranks", "w") as f:
    header = "\t".join(vocabs.keys())
    f.write(header + "\n")
    for token, ranks in token_to_ranks.items():
        escaped_token = escape_token(token)
        f.write(escaped_token + "\t" + "\t".join(map(str, ranks)) + "\n")